**Problema.**
La concentración del producto

dt/dC​=kC(t)(Cmax​−C(t)),

con condiciones y parámetros:

*   C(0)=0.02 mol·L−1
*  k=0.45 L·mol−1·min−1
*   Cmax=1.0 mol·L−1
*   intervalo de integración:t∈[0,25] min

Objetivo.Predecir
C(t) en 0–25 min usando Runge–Kutta de 4º orden (RK4) con
Δt=0.1, comparar con Runge–Kutta de 2º orden (RK2, Heun) con el mismo Δt, y determinar qué Δt debe usar RK2 para obtener resultados equivalentes.


In [9]:
%%writefile logistica_rk.c
/* logistica_rk.c
   Solucion de la EDO logistica usando RK2 (Heun) y RK4.
   dC/dt = k * C * (Cmax - C)
*/

#include <stdio.h>
#include <math.h>   // <-- ESTA LÍNEA QUITA LOS WARNINGS

double k = 0.45;
double Cmax = 1.0;

/* dC/dt */
double f(double C) {
    return k * C * (Cmax - C);
}

/* RK4 */
double rk4_step(double C, double h) {
    double k1 = f(C);
    double k2 = f(C + 0.5*h*k1);
    double k3 = f(C + 0.5*h*k2);
    double k4 = f(C + h*k3);
    return C + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4);
}

/* RK2 (Heun) */
double rk2_heun_step(double C, double h) {
    double k1 = f(C);
    double k2 = f(C + h*k1);
    return C + 0.5*h*(k1 + k2);
}

int main() {
    double C0 = 0.02;
    double h = 0.1;
    double T = 25.0;
    int N = (int)(T / h);

    double Ct_rk4 = C0;
    double Ct_rk2 = C0;
    double t = 0.0;

    printf("t(min)\t\tRK4_C(t)\t\tRK2_C(t)\t\t|diff|\n");
    printf("--------------------------------------------------------------\n");

    printf("%.2f\t\t%.9f\t%.9f\t%.9f\n", t, Ct_rk4, Ct_rk2, fabs(Ct_rk4 - Ct_rk2));

    for (int i = 1; i <= N; i++) {
        Ct_rk4 = rk4_step(Ct_rk4, h);
        Ct_rk2 = rk2_heun_step(Ct_rk2, h);
        t = i * h;

        printf("%.2f\t\t%.9f\t%.9f\t%.9f\n",
               t, Ct_rk4, Ct_rk2, fabs(Ct_rk4 - Ct_rk2));
    }

    return 0;
}


Overwriting logistica_rk.c


In [10]:
!gcc logistica_rk.c -o logistica_rk -lm

In [11]:
!./logistica_rk > salida_logistica.txt
!sed -n '1,30p' salida_logistica.txt

t(min)		RK4_C(t)		RK2_C(t)		|diff|
--------------------------------------------------------------
0.00		0.020000000	0.020000000	0.000000000
0.10		0.020901316	0.020901034	0.000000283
0.20		0.021842346	0.021841757	0.000000589
0.30		0.022824755	0.022823834	0.000000921
0.40		0.023850273	0.023848993	0.000001281
0.50		0.024920693	0.024919025	0.000001668
0.60		0.026037873	0.026035787	0.000002086
0.70		0.027203737	0.027201202	0.000002535
0.80		0.028420281	0.028417263	0.000003018
0.90		0.029689568	0.029686032	0.000003536
1.00		0.031013734	0.031009642	0.000004092
1.10		0.032394986	0.032390300	0.000004686
1.20		0.033835606	0.033830285	0.000005322
1.30		0.035337952	0.035331952	0.000006000
1.40		0.036904456	0.036897732	0.000006724
1.50		0.038537628	0.038530133	0.000007495
1.60		0.040240054	0.040231739	0.000008315
1.70		0.042014400	0.042005213	0.000009188
1.80		0.043863409	0.043853295	0.000010114
1.90		0.045789901	0.045778804	0.000011097
2.00		0.047796775	0.047784636	0.000012139
2.10		0.049887007	0.

In [12]:
!grep -n "^25.00" salida_logistica.txt || true
!grep "25.00" salida_logistica.txt || true


253:25.00		0.999363048	0.999361235	0.000001813
5.60		0.202325900	0.202221745	0.000104155
25.00		0.999363048	0.999361235	0.000001813


 **¿Qué tamaño de Δt necesita RK2 para “igualar” RK4 (con
ℎ=0.1)?**
Para que la máxima diferencia entre RK2 y RK4 sea ≤ 10−4: usar
Δt≤0.05.

Para diferencia ≤10−6: usar Δt≈0.01.

En la práctica, con Δt=0.05 RK2 muestra errores comparables al RK4 con Δt=0.1 para esta EDO.